# 2.2 Learning Models and Keras Part 1

## Index
### Import Libraries and Data
### Clean Data
### Reshape Data
### Split Data
### Create Keras Model
### Compile and run RNN Model
### Confusion Matrix of Model
### Adjust Hyperparameters
### Adjust Hyperparameters and changing Activation
### Adjust Hyperparameters, adding convolution and pooling layers, changing optimizer
### Hyperparameters2

## Import Libraries and Data

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
path = r'C:/Users/Jack Bartman/OneDrive/CareerFoundry/Machine Learning'

In [9]:
# Import data
climate =pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'DATASET weather_prediction_dataset_processed.csv'))

In [11]:
pleasant =pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

## Clean Data

In [14]:
climate.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [16]:
# drop date and month from climate
climate = climate.drop(columns=['DATE', 'MONTH'], axis=1)
climate.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [18]:
# Drop all Gdansk, Roma,Tours columns from df_unscaled since they are not included in pleasant weather data
cols_to_drop = [col for col in climate.columns if col.startswith(('GDANSK', 'ROMA', 'TOURS'))]
climate = climate.drop(columns=cols_to_drop)

In [20]:
climate.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [22]:
# Drop date column from df_pleasant
pleasant = pleasant.drop(columns=['DATE'], axis=1)
pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# find out all the different measurement types for each location

# Extract location names 
locations = set([col.split('_')[0] for col in climate.columns])

# Create a dictionary to store measurement counts for each location
measurement_counts = {location: {} for location in locations}

# Count occurrences of each measurement type for each location
for col in climate.columns:
    parts = col.split('_') 
    location = parts[0] 
    measurement = '_'.join(parts[1:])  # Join remaining parts if there are more than two

    if measurement not in measurement_counts[location]:
        measurement_counts[location][measurement] = 1
    else:
        measurement_counts[location][measurement] += 1

# Print the measurement counts for each location
for location, measurements in measurement_counts.items():
    print(f"Location: {location}")
    for measurement, count in measurements.items():
        print(f"  - {measurement}: {count}")
    print()

Location: BELGRADE
  - cloud_cover: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: BASEL
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: KASSEL
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: MAASTRICHT
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: MUNCHENB
  - cloud_cover: 1
  - humidity: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: MADRID
  - cloud_cover: 1
  - win

In [30]:
## Missing
## Maastricht: snow_depth
## Madrid: snow_depth
## Heathrow: wind_speed measurements
## Kassel: cloud_cover, snow_depth
## Belgrade: wind_speed, snow_depth
## Valentia: wind_speed
## Budapest: wind_speed, snow_depth
## Ljubljana: snow_depth
## Sonnblick: snow_depth
## Stockholm: wind_speed, humidity, snow_depth
## Debilt: snow_depth
## Munchenb wind_speed, pressure

In [32]:
# Drop columns for wind_speed and snow_depth
cols_to_drop = [col for col in climate.columns if col.endswith(('wind_speed', 'snow_depth'))]
climate = climate.drop(columns=cols_to_drop)

In [34]:
climate.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [38]:
# There are missing measurements for Kassel's cloud cover, Stockholm's humidity, and Munchenb's pressure
# We know that Ljubljana is near Kassel, Sonnblick is near Munchenb, and Olso is near enough to Stockholm

# Define relationships between locations
location_pairs = {
    'KASSEL': 'LJUBLJANA',
    'STOCKHOLM': 'OSLO',
    'MUNCHENB': 'SONNBLICK'
}

# Define the desired order of measurements
measurement_order = ['cloud_cover', 'humidity', 'pressure', 'global_radiation', 
                     'precipitation', 'sunshine', 'temp_mean', 'temp_min', 'temp_max']

# Function to fill missing values and insert in correct position
def fill_missing_values(climate, location, measurement, neighbor):
    """
    Fills missing values for a given location and measurement using data from a neighbor location.
    Inserts the new column in the correct position based on the measurement order.

    Args:
        climate: The DataFrame containing the weather data.
        location: The location with missing values.
        measurement: The measurement with missing values.
        neighbor: The neighboring location to use for filling.

    Returns:
        The updated DataFrame with filled missing values and columns in the correct order.
    """
    source_col = f'{neighbor}_{measurement}'
    target_col = f'{location}_{measurement}'

    # Determine the insertion index 
    if measurement == measurement_order[0]:  # If it's the first measurement for the location
        # Find the index of the first column for the location (or 0 if no location columns exist)
        location_columns = [col for col in climate.columns if col.startswith(location)]
        if location_columns:
            insert_index = climate.columns.get_loc(location_columns[0]) 
        else:
            insert_index = 0
    else:
        insert_index = climate.columns.get_loc(f'{location}_{measurement_order[measurement_order.index(measurement) - 1]}') + 1 

    # Create the new column with missing values and insert it at the correct position
    climate.insert(insert_index, target_col, np.nan) 

    # Fill missing values in the new column
    climate[target_col].fillna(climate[source_col], inplace=True) 

    return climate

# Fill missing values for each location and measurement
for location, neighbor in location_pairs.items():
    for measurement in measurement_order:
        if f'{location}_{measurement}' not in climate.columns:  # Check if column already exists
            climate = fill_missing_values(climate, location, measurement, neighbor)

# Checking new columns for existance and location
selected_columns = [col for col in climate.columns if col.startswith(('KASSEL', 'STOCKHOLM', 'MUNCHENB'))]
print(climate[selected_columns])

       KASSEL_cloud_cover  KASSEL_humidity  KASSEL_pressure  \
0                     8.0             0.82           1.0094   
1                     6.0             0.86           1.0086   
2                     8.0             0.91           1.0129   
3                     6.0             0.87           1.0290   
4                     7.0             0.86           1.0262   
...                   ...              ...              ...   
22945                 4.0             0.77           1.0161   
22946                 3.0             0.77           1.0161   
22947                 3.0             0.77           1.0161   
22948                 3.0             0.77           1.0161   
22949                 3.0             0.77           1.0161   

       KASSEL_global_radiation  KASSEL_precipitation  KASSEL_sunshine  \
0                         0.28                  0.48              1.6   
1                         0.12                  0.27              0.0   
2                       

C:\Users\Jack Bartman\AppData\Local\Temp\ipykernel_14404\1886491939.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  climate[target_col].fillna(climate[source_col], inplace=True)
C:\Users\Jack Bartman\AppData\Local\Temp\ipykernel_14404\1886491939.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [40]:
# Checking new columns for existance and location
selected_columns = [col for col in climate.columns if col.startswith(('MUNCHENB'))]
print(climate[selected_columns])

       MUNCHENB_cloud_cover  MUNCHENB_humidity  MUNCHENB_pressure  \
0                         5               0.67             1.0304   
1                         6               0.72             1.0292   
2                         6               0.91             1.0320   
3                         6               0.90             1.0443   
4                         5               0.85             1.0430   
...                     ...                ...                ...   
22945                     2               0.76             1.0263   
22946                     6               0.70             1.0263   
22947                     7               0.64             1.0263   
22948                     6               0.75             1.0263   
22949                     5               0.83             1.0263   

       MUNCHENB_global_radiation  MUNCHENB_precipitation  MUNCHENB_sunshine  \
0                           0.20                    0.10                0.0   
1            

In [42]:
climate.shape

(22950, 135)

In [48]:
pleasant.shape

(22950, 15)

In [52]:
# Export cleaned weather data
climate.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'weather_clean.csv'), index=False)

## Reshape Data

In [55]:
# Rename df's
X = climate
y = pleasant

In [57]:
# Convert df's to arrays
X = np.array(X)
y = np.array(y)

In [59]:
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [61]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [63]:
# Reshaping X as a 3D object
X = X.reshape(-1,15,9)

In [65]:
X.shape

(22950, 15, 9)

In [67]:
y.shape

(22950, 15)

## Split Data

In [70]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

## Create Keras Model

In [73]:
epochs = 20
batch_size = 16
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\anaconda\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [75]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 16)             │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,919 (7.50 KB)

 Trainable params: 1,919 (7.50 KB)

 Non-trainable params: 0 (0.00 B)

## Compile and Run RNN Model

In [78]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [80]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1045 - loss: 10.0289 - val_accuracy: 0.1842 - val_loss: 8.4589
Epoch 2/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1975 - loss: 10.0462 - val_accuracy: 0.2623 - val_loss: 8.5934
Epoch 3/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2735 - loss: 9.6848 - val_accuracy: 0.3116 - val_loss: 8.8989
Epoch 4/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2879 - loss: 10.1991 - val_accuracy: 0.3041 - val_loss: 9.1845
Epoch 5/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2953 - loss: 10.3659 - val_accuracy: 0.3135 - val_loss: 9.5264
Epoch 6/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3297 - loss: 10.4296 - val_accuracy: 0.3060 - val_loss: 9.7929
Epoch 7/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4037 - loss: 10.7235 - val_accuracy: 0.6389 - val_loss: 10.0326
Epoch 8/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5053 - loss: 11

## Confusion Marix of Models

In [84]:
# Define list of stations names
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [86]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [88]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        3676       6
BELGRADE     1090       2
BUDAPEST      212       2
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       82       0
KASSEL         11       0
LJUBLJANA      59       2
MAASTRICHT      9       0
MADRID        451       7
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


## Adjust Hyperparameters

In [91]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\anaconda\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [93]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,871 (22.93 KB)

 Trainable params: 5,871 (22.93 KB)

 Non-trainable params: 0 (0.00 B)

In [95]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [97]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1304 - loss: 10.2663 - val_accuracy: 0.2684 - val_loss: 9.0097
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2745 - loss: 10.7839 - val_accuracy: 0.2734 - val_loss: 9.3033
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2754 - loss: 10.8235 - val_accuracy: 0.2701 - val_loss: 9.3311
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3761 - loss: 10.8919 - val_accuracy: 0.6392 - val_loss: 9.6561
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5642 - loss: 10.7314 - val_accuracy: 0.6424 - val_loss: 10.0070
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6251 - loss: 11.1447 - val_accuracy: 0.6398 - val_loss: 10.0619
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6409 - loss: 10.9196 - val_accuracy: 0.6413 - val_loss: 10.3199
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6360 - loss:

In [99]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [101]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        3680       2
BELGRADE     1092       0
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       82       0
KASSEL         11       0
LJUBLJANA      61       0
MAASTRICHT      9       0
MADRID        455       3
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


## Adjusting Hyperparameters and changing Activation

In [104]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

C:\anaconda\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [106]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,919 (77.81 KB)

 Trainable params: 19,919 (77.81 KB)

 Non-trainable params: 0 (0.00 B)

In [108]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [110]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.0338 - loss: 25.1978 - val_accuracy: 0.0899 - val_loss: 23.6444
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1118 - loss: 24.9946 - val_accuracy: 6.9711e-04 - val_loss: 22.2874
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0218 - loss: 24.9241 - val_accuracy: 6.9711e-04 - val_loss: 25.3505
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0123 - loss: 24.8415 - val_accuracy: 0.0371 - val_loss: 26.4648
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0352 - loss: 25.5090 - val_accuracy: 0.0373 - val_loss: 25.9370
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0427 - loss: 25.0347 - val_accuracy: 0.0082 - val_loss: 26.1320
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0433 - loss: 25.1249 - val_accuracy: 0.0112 - val_loss: 27.8696
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.

In [112]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [114]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BELGRADE  DEBILT  HEATHROW  LJUBLJANA  MADRID  OSLO
True                                                                  
BASEL         109      2030      11         1        216     194  1121
BELGRADE       10       302       0         0          0      60   720
BUDAPEST        2        97       0         0          0       7   108
DEBILT          0        23       0         0          0       0    59
DUSSELDORF      0        10       0         0          0       0    19
HEATHROW        0        55       0         0          0       0    27
KASSEL          0         5       0         0          0       0     6
LJUBLJANA       0        50       0         0          0       1    10
MAASTRICHT      0         4       0         0          0       0     5
MADRID          3       350       0         0          1      13    91
MUNCHENB        0         4       0         0          0       0     4
OSLO            0         4       0 

## Adjust Hyperparameters, adding convolution and pooling layers, changing optimizer

In [117]:
epochs = 25
batch_size = 16
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

C:\anaconda\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\anaconda\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [119]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 8)              │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 831 (3.25 KB)

 Trainable params: 831 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

In [121]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [123]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0351 - loss: 21.3744 - val_accuracy: 0.0739 - val_loss: 30.2837
Epoch 2/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0307 - loss: 24.7237 - val_accuracy: 0.0364 - val_loss: 32.3106
Epoch 3/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0289 - loss: 29.2903 - val_accuracy: 0.0124 - val_loss: 38.9583
Epoch 4/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0163 - loss: 28.6211 - val_accuracy: 0.0063 - val_loss: 29.1014
Epoch 5/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0093 - loss: 26.5345 - val_accuracy: 0.0063 - val_loss: 23.5829
Epoch 6/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0190 - loss: 26.8307 - val_accuracy: 0.0448 - val_loss: 32.5246
Epoch 7/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0256 - loss: 24.3280 - val_accuracy: 0.0080 - val_loss: 24.9080
Epoch 8/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0203 - l

In [125]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [127]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BUDAPEST  DUSSELDORF  HEATHROW  LJUBLJANA  MAASTRICHT  MADRID  \
True                                                                        
BASEL           1248         441         1        386         317       1   
BELGRADE         618         295         0         82           8       0   
BUDAPEST         149          31         0         20           0       0   
DEBILT            68          10         0          4           0       0   
DUSSELDORF        20           4         0          3           0       0   
HEATHROW          47           9         0         14           0       0   
KASSEL             8           1         0          0           0       0   
LJUBLJANA         32           4         0          9           0       0   
MAASTRICHT         5           3         0          1           0       0   
MADRID           152          44         0         64           2       0   
MUNCHENB           2           4   

## Hyperparameters2

In [130]:
epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\anaconda\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [132]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 4)              │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 299 (1.17 KB)

 Trainable params: 299 (1.17 KB)

 Non-trainable params: 0 (0.00 B)

In [134]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [136]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.0881 - loss: 8.7642 - val_accuracy: 0.0793 - val_loss: 8.4874
Epoch 2/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.1613 - loss: 9.0643 - val_accuracy: 0.3099 - val_loss: 9.3844
Epoch 3/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.2571 - loss: 10.0929 - val_accuracy: 0.3083 - val_loss: 10.0768
Epoch 4/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.2862 - loss: 10.5610 - val_accuracy: 0.3118 - val_loss: 10.7365
Epoch 5/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.2811 - loss: 11.1254 - val_accuracy: 0.3130 - val_loss: 11.2042
Epoch 6/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.2767 - loss: 11.6994 - val_accuracy: 0.3158 - val_loss: 11.7429
Epoch 7/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.3291 - loss: 12.3461 - val_accuracy: 0.3132 - val_loss: 12.2034
Epoch 8/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.3574 - loss

In [140]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [142]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  MADRID
True                     
BASEL        1555    2127
BELGRADE     1030      62
BUDAPEST      210       4
DEBILT         82       0
DUSSELDORF     28       1
HEATHROW       69      13
KASSEL         11       0
LJUBLJANA      55       6
MAASTRICHT      5       4
MADRID        203     255
MUNCHENB        7       1
OSLO            4       1
STOCKHOLM       4       0
VALENTIA        0       1
